In [1]:
import pickle
import pandas as pd

In [2]:
with open('scrapes/setlists_2020-12-16-T1733.pickle', 'rb') as handle:
    shows = pickle.load(handle)

In [3]:
name = []
stewart = []
in_trans = []
out_trans = []
for show in shows:
    setlists = show['sets']
    if setlists is not None:
        names = []
        for setlist in setlists:
            for song in setlists[setlist]:
                names.append(song['name'])
                stewart.append(song['stewart'])
                in_trans.append(song['in_transition'])
                out_trans.append(song['out_transition'])
        name = name + list(set(names))

In [4]:
# Create CSVs
shows = pd.DataFrame(shows)
songs = pd.read_csv('scrapes/songs_2020-12-16-T1733.csv', index_col=0)

In [5]:
# How off are the counts?
(pd.Series(name).value_counts() - songs['times_played']).value_counts()

 0.0    892
-1.0     41
-2.0     13
-3.0      1
 1.0      1
dtype: int64

In [6]:
# Songs in setlist but not listed in stats
missing = []
for x in name:
    if x not in list(songs.index):
        missing.append(x)
missing = list(set(missing))
missing

['Improvisation',
 'Hair Band Medley',
 'bass jam',
 'Halloween Medley',
 'unknown song',
 'OM Trio Jam',
 'piano improv',
 'drum solo',
 'Medley',
 'Irish Jig',
 'Drum solo',
 'XRT Xmas Promo Medley',
 '"Horns vs. Band Challenge"',
 '"Jimmy Stewart"',
 '"Get Weird" JO',
 'Jazz Odyssey',
 'Jimmy Stewart',
 'Jamiroquai #7',
 '"Zappa (Jake conducts) improv"',
 'piano solo',
 'Percussion']

In [7]:
# Songs in stats but not listed in setlist 
missing = []
for x in list(songs.index):
    if x not in name:
        missing.append(x)
missing = list(set(missing))
missing

[]

In [8]:
display(shows.head())
display(songs.head())

,date,title,sets,reviews,support,notes,tags
0,01.17.2020,"Beacon Theatre, New York, NY, USA","{'Set 1': [{'name': 'Unsung Hero', 'stewart': ...",None,None,[Anthem was in remembrance of Neil Peart],{1: 'began with More Than Words (Extreme) teas...
1,01.18.2020,"Beacon Theatre, New York, NY, USA","{'Set 1': [{'name': 'End of the Road', 'stewar...",None,None,[Limelight was in remembrance of Neil Peart],"{1: 'with Bright Lights, Big City jam', 2: 'Ai..."
2,01.19.2020,"Brooklyn Bowl, Brooklyn, NY, USA","{'Set 1': [{'name': 'Looks', 'stewart': None, ...",None,None,None,"{1: 'began with YYZ (Rush) tease', 2: 'with Q*..."
3,01.23.2020,"Penn's Peak, Jim Thorpe, PA, USA","{'Set 1': [{'name': 'Plunger', 'stewart': None...",None,The New Deal,None,"{1: 'with Rhiannon (Fleetwood Mac) teases', 2:..."
4,01.24.2020,"Stage AE, Pittsburgh, PA, USA","{'Set 1': [{'name': 'Depth Charge', 'stewart':...",None,The New Deal,[last Tom Sawyer 02.18.2017 (264 shows)],"{1: 'debut, original; with Jake on acoustic', ..."


,artist,times_played,debut_date,last_played
...And Justice for All,Metallica,28,2003.10.31,2020.03.06
...And We Became Sunshine,The Uglysuit,1,2010.09.25,2010.09.25
"""Brendan & Jake switch rigs""",Umphrey's McGee,2,2010.04.24,2014.05.03
"""Mrs Robinson's Strut""",Umphrey's McGee,1,2011.04.02,2011.04.02
(Don't Fear) The Reaper,Blue Öyster Cult,14,2001.10.31,2013.08.15


## Song Abbreviations

In [9]:
def remove_vowels(string):
    return (string.replace('a', '')
              .replace('e', '')
              .replace('i', '')
              .replace('o', '')
              .replace('u', ''))

def abbrv_word(word):
    if len(word) < 8 and word[:4] not in list(song_to_abbrv.values()):
        word = word[:4]
    else:
        if len(remove_vowels(word)[:4]) < 4:
            word = word[:4]
        else:
            word = remove_vowels(word)[:4]
    return word

def try_word(word):
    if len(word) < 6 and len(word) >= 3:
        abbrv = abbrv_word(word)
        if abbrv not in list(song_to_abbrv.values()):
            return abbrv

In [10]:
song_to_abbrv = {}
abbrv_to_song = {}
for song in list(songs.index):
    abbrv = song.lower()
    abbrv = (abbrv.replace('-', '')
                  .replace("'", '')
                  .replace(",", '')
                  .replace(":", '')
                  .replace(".", '')
                  .replace("*", '')
                  .replace("(", '')
                  .replace(")", '')
                  .replace('"', '')
                  .replace("#", 'n')
                  .replace("i ", '')
                  .replace("to ", '')
                  .replace("a ", '')
                  .replace("the ", '')
                  .replace("an ", ''))
    artist = "Umphrey's McGee" if song not in list(songs.index) else songs.loc[song]['artist']
    if artist == "Umphrey's McGee":
        abbrv = abbrv.upper()
    else:
        abbrv = abbrv.lower()
              
    words = abbrv.split(' ')
    if len(words) == 1:    
        abbrv = abbrv_word(words[0])
    elif len(words) == 2:  
        abbrv = try_word(words[0])
        abbrv = try_word(words[1])
        if abbrv is None:
            abbrv = words[0][:2] + words[1][:2]  
    elif len(words) == 3:       
        abbrv = try_word(words[0])
        abbrv = try_word(words[1])
        abbrv = try_word(words[2])
        if abbrv is None:
            abbrv = words[0][:2] + words[1][0] + words[2][0]  
    elif len(words) > 3:      
        abbrv = ''.join([x[0] for x in words][:4])  
    else: 
        abbrv = None    
    
    if abbrv in list(song_to_abbrv.values()) and song.lower() == abbrv.lower():
        tmp = abbrv
        prev_song = abbrv_to_song[tmp]
        while tmp in list(song_to_abbrv.values()):
            tmp = input('%s, %s:' % (prev_song, tmp))
            tmp = tmp.upper() if artist == "Umphrey's McGee" else tmp.lower()
        song_to_abbrv[prev_song] = tmp
        abbrv_to_song[tmp] = prev_song
    
    
    while abbrv in list(song_to_abbrv.values()):
        abbrv = input('%s, %s, %s: ' % (song, artist, abbrv))
        abbrv = abbrv.upper() if artist == "Umphrey's McGee" else abbrv.lower()
        
    song_to_abbrv[song] = abbrv
    abbrv_to_song[abbrv] = song

Another Brick in the Wall/Thriller, Pink Floyd/Michael Jackson, abiw: abth
Gut Strut, Umphrey's McGee, GUST: guts
Hours, Umphrey's McGee, HOUR: hors
Funk Jam, JAM:funj
Soul Food II, Ali Baba's Tahini, sofi: sof2
The Star Spangled Banner, Francis Scott Key, stsb: stsp
The Stranger, Billy Joel, strn: strg
November Walk, walk:nwlk
Walletsworth, Umphrey's McGee, WALL: walw
Whitehouse Road, Tyler Childers, whro: whtr
Words, Umphrey's McGee, WORD: wrds


In [11]:
# PAST RUN
# Another Brick in the Wall/Thriller, Pink Floyd/Michael Jackson, abiw: abth
# Gut Strut, Umphrey's McGee, GUST: guts
# Hours, Umphrey's McGee, HOUR: hors
# Funk Jam, JAM:funj
# Soul Food II, Ali Baba's Tahini, sofi: sof2
# The Star Spangled Banner, Francis Scott Key, stsb: stsp
# The Stranger, Billy Joel, strn: strg
# November Walk, walk:nwlk
# Walletsworth, Umphrey's McGee, WALL: walw
# Whitehouse Road, Tyler Childers, whro: whtr
# Words, Umphrey's McGee, WORD: wrds

In [12]:
# Ensure unique
a = len(song_to_abbrv)
b = len(list(set(song_to_abbrv.values())))
print(a,b)
count = pd.Series(list(song_to_abbrv.values())).value_counts().to_dict()
[x for x in count if count[x] > 1]
assert a == b

948 948


In [13]:
songs['code'] = pd.Series(song_to_abbrv)
songs = songs.reset_index().rename(columns={'index':'name'}).set_index('code')

In [14]:
songs

,name,artist,times_played,debut_date,last_played
code,,,,,
ajfa,...And Justice for All,Metallica,28,2003.10.31,2020.03.06
awbs,...And We Became Sunshine,The Uglysuit,1,2010.09.25,2010.09.25
BJSR,"""Brendan & Jake switch rigs""",Umphrey's McGee,2,2010.04.24,2014.05.03
STRU,"""Mrs Robinson's Strut""",Umphrey's McGee,1,2011.04.02,2011.04.02
dofr,(Don't Fear) The Reaper,Blue Öyster Cult,14,2001.10.31,2013.08.15
...,...,...,...,...,...
316,316,Van Halen,4,2012.01.10,2015.08.21
1PSB,1000 Places to See Before You Die,Umphrey's McGee,7,2007.03.24,2019.02.23
1348,1348,Umphrey's McGee,239,2009.01.23,2020.09.05


In [79]:
import collections
codes = list(collections.OrderedDict(sorted(song_to_abbrv.items())).items())

In [80]:
table = {}
for i in range(100):
    table[i] = []
for i in range(len(codes)):
    table[i % 100].append(codes[i])
for i in range(100):
    table[i] = [item for sublist in table[i] for item in sublist]
table = list(table.values())

In [81]:
pd.DataFrame(table).to_csv('song_codes.csv')